# Project 5- Modeling (casualties)

This notebook is to create a model that will predict civilian casualties of the Yemen conflict, when knowing the
location of an air raid.  For this problem we will build a Regressive Neural Network to model the area data provided by
the Yemen Data Project.

#### Import packages

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

%matplotlib inline 
plt.style.use('fivethirtyeight')

#### Read in Data

In [2]:
df = pd.read_csv('data/clean_df.csv')

In [3]:
df.head()

date  incident_id governorate        district  \
0  2015-03-26            1       Sanaa  bani al-harith   
1  2015-03-26            2       Sanaa  bani al-harith   
2  2015-03-26            3       Sanaa  bani al-harith   
3  2015-03-26            4       Sanaa      al-sab'ein   
4  2015-03-26            5       Sanaa      al-thawrah   

                       area                        target  \
0                Al-Rahabah            Al-Daylami Airbase   
1                   Airport  Sana'a International Airport   
2  Al-Sonblah Neighbourhood              Residential Area   
3                Al-Nahdain           Presidential Palace   
4                Al-Nahdhah  Former 1st Armoured Division   

              main_category      sub_category  min_air_raids  max_air_raids  \
0  military_security_target     military site              1            2.0   
1            infrastructure         transport              1            2.0   
2                  civilian  residential area              1            2.0   
3  military_security_target     military site              1            2.0   
4  military_security_target     military site              1            2.0   

   civilian_casualties  fatalities  woman_fatalities  child_fatalities  \
0                    0           0                 0                 0   
1                    0           0                 0                 0   
2                   29          21                 3                14   
3                    0           0                 0                 0   
4                    0           0                 0                 0   

   injured  woman_injured  child_injured confirmed_time time_of_day  
0        0              0            0.0              2     morning  
1        0              0            0.0              2     morning  
2        8              0            7.0              2     morning  
3        0              0            0.0              3     morning  
4        0              0            0.0              3     morning

#### Resetting index to datetime index

In [4]:
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index) 

In [5]:
df.columns

Index(['incident_id', 'governorate', 'district', 'area', 'target',
       'main_category', 'sub_category', 'min_air_raids', 'max_air_raids',
       'civilian_casualties', 'fatalities', 'woman_fatalities',
       'child_fatalities', 'injured', 'woman_injured', 'child_injured',
       'confirmed_time', 'time_of_day'],
      dtype='object')

#### Dummify columns with text

We are dummying these columns to extract the data and make the data numerically represented.

In [6]:
df = pd.get_dummies(df, columns=['main_category', 'sub_category', 'time_of_day', 
                    'confirmed_time', 'governorate', 'district', 'area', 'target'])

In [7]:
df.head()

incident_id  min_air_raids  max_air_raids  civilian_casualties  \
date                                                                         
2015-03-26            1              1            2.0                    0   
2015-03-26            2              1            2.0                    0   
2015-03-26            3              1            2.0                   29   
2015-03-26            4              1            2.0                    0   
2015-03-26            5              1            2.0                    0   

            fatalities  woman_fatalities  child_fatalities  injured  \
date                                                                  
2015-03-26           0                 0                 0        0   
2015-03-26           0                 0                 0        0   
2015-03-26          21                 3                14        8   
2015-03-26           0                 0                 0        0   
2015-03-26           0                 0                 0        0   

            woman_injured  child_injured  ...  target_three hotels  \
date                                      ...                        
2015-03-26              0            0.0  ...                    0   
2015-03-26              0            0.0  ...                    0   
2015-03-26              0            7.0  ...                    0   
2015-03-26              0            0.0  ...                    0   
2015-03-26              0            0.0  ...                    0   

            target_three vehicles  target_two military vehicles  \
date                                                              
2015-03-26                      0                             0   
2015-03-26                      0                             0   
2015-03-26                      0                             0   
2015-03-26                      0                             0   
2015-03-26                      0                             0   

            target_unknown  target_unknown   target_vocational Institute  \
date                                                                       
2015-03-26               0                0                            0   
2015-03-26               0                0                            0   
2015-03-26               0                0                            0   
2015-03-26               0                0                            0   
2015-03-26               0                0                            0   

            target_water drill  target_water truck  target_water well  \
date                                                                    
2015-03-26                   0                   0                  0   
2015-03-26                   0                   0                  0   
2015-03-26                   0                   0                  0   
2015-03-26                   0                   0                  0   
2015-03-26                   0                   0                  0   

            target_well  
date                     
2015-03-26            0  
2015-03-26            0  
2015-03-26            0  
2015-03-26            0  
2015-03-26            0  

[5 rows x 9084 columns]

In [8]:
df.columns

Index(['incident_id', 'min_air_raids', 'max_air_raids', 'civilian_casualties',
       'fatalities', 'woman_fatalities', 'child_fatalities', 'injured',
       'woman_injured', 'child_injured',
       ...
       'target_three hotels', 'target_three vehicles',
       'target_two military vehicles', 'target_unknown', 'target_unknown ',
       'target_vocational Institute', 'target_water drill',
       'target_water truck', 'target_water well', 'target_well'],
      dtype='object', length=9084)

## Setting up and scaling Training and Testing data 

Create X and y:  We had to remove multiple features from X as they would have leaked our target value, civilian
casualties.  This fits our scenario, as when a report comes in given the location of the raid, we want to predict
casualties without reports of casualty specifics.

In [21]:
X = df.drop(columns=['civilian_casualties', 'fatalities', 'woman_fatalities',
       'child_fatalities', 'injured', 'woman_injured', 'child_injured'])
y = df['civilian_casualties'] # Predicting casualties


Split data into training and testing datasets.

In [10]:
# Shuffle set to false on Train/Test split because it's a time series index
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, shuffle=False)

Apply a standard scalar to our data.  This will regualirze the result and make our model more accurtate.

In [14]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

### Creating a Regressive Neural Network to predict civilian casualties

Create variable input_shape to be used as a parameter in the model.

In [15]:
input_shape = X_train_ss[0].shape


This model will have 8 layers, one input and one output layer, and 6 hidden layers.

In [19]:
model = Sequential()

model.add(Dense(256, input_shape=input_shape, activation="relu")) #Input layer with 256 neurons

model.add(Dense(128, activation="relu")) # 6 Hidden Layers

model.add(Dense(128, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(16, activation='relu'))

model.add(Dense(8, activation='relu'))

model.add(Dense(1)) #because this is a regression prediction we only use 1 node in the last output layer

model.compile(optimizer='adam', loss='mse', metrics=['acc']) # using MSE as metric for regression

history = model.fit(X_train_ss, y_train, validation_data=(X_test_ss, y_test), epochs=100, batch_size = 512)

Epoch 1/100
33/33 [==============================] - 3s 76ms/step - loss: 75.0530 - acc: 0.6683 - val_loss: 22.9074 - val_acc: 0.3595
Epoch 2/100
33/33 [==============================] - 2s 50ms/step - loss: 67.1621 - acc: 0.4609 - val_loss: 22.6593 - val_acc: 0.6869
Epoch 3/100
33/33 [==============================] - 2s 49ms/step - loss: 40.3396 - acc: 0.7662 - val_loss: 23.6470 - val_acc: 0.8049
Epoch 4/100
33/33 [==============================] - 2s 46ms/step - loss: 58.1601 - acc: 0.8004 - val_loss: 23.4195 - val_acc: 0.9248
Epoch 5/100
33/33 [==============================] - 2s 50ms/step - loss: 105.0168 - acc: 0.7921 - val_loss: 23.4042 - val_acc: 0.8908
Epoch 6/100
33/33 [==============================] - 2s 48ms/step - loss: 91.3043 - acc: 0.8042 - val_loss: 23.0552 - val_acc: 0.7303
Epoch 7/100
33/33 [==============================] - 2s 47ms/step - loss: 16.8559 - acc: 0.7874 - val_loss: 29.0225 - val_acc: 0.8504
Epoch 8/100
33/33 [==============================] - 2s 49ms/

Epoch 62/100
33/33 [==============================] - 1s 44ms/step - loss: 32.9455 - acc: 0.8864 - val_loss: 27.9037 - val_acc: 0.8812
Epoch 63/100
33/33 [==============================] - 2s 46ms/step - loss: 37.9941 - acc: 0.8956 - val_loss: 26.5565 - val_acc: 0.8678
Epoch 64/100
33/33 [==============================] - 2s 53ms/step - loss: 22.3769 - acc: 0.8893 - val_loss: 30.3432 - val_acc: 0.8742
Epoch 65/100
33/33 [==============================] - 2s 48ms/step - loss: 34.0736 - acc: 0.8957 - val_loss: 27.5916 - val_acc: 0.8751
Epoch 66/100
33/33 [==============================] - 2s 47ms/step - loss: 86.4006 - acc: 0.8975 - val_loss: 26.2922 - val_acc: 0.8593
Epoch 67/100
33/33 [==============================] - 1s 45ms/step - loss: 6.1558 - acc: 0.8979 - val_loss: 31.5217 - val_acc: 0.8627
Epoch 68/100
33/33 [==============================] - 2s 46ms/step - loss: 35.0710 - acc: 0.8943 - val_loss: 25.1934 - val_acc: 0.8855
Epoch 69/100
33/33 [==============================] - 2s

## Plotting our Training/Testing Loss and Accuracy


#### Loss

In [32]:
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over number of Epochs')
plt.legend();

Our loss values for training and testing leveled off at 5.0957 and 27.1714 respectively after 100 epochs, although this
value tended to jump back and forth throughout all the epochs.  While there is little gain from 50 epochs to 100, there
is a trend downward showing the loss improving as the model reaches 100 epochs.



#### Accuracy

In [33]:
plt.plot(history.history['acc'], label='Train accuracy')
plt.plot(history.history['val_acc'], label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over number of Epochs')
plt.legend();

Our accuracy score ended at .8988 for training and .8945 for testing, so our model was not overfit to the training data,
 however the value jumped back and forth around .87-.88.  This model may improve slightly with more epoch but the model's,
 accuracy will not be that much different as both the training and testing data have reached a theoretical asymptote of .9.

## Overall

We created a regressive nueral network to predict civilian casualties when given the location of an air raid.  Our model is 89.45% accurate in estimating civilian casuatlies when provided data on where an air raid took place. This
model will help recommend how much aid to allocate towards medical resources, funding, and personnel based on location of
 the attack.  This model would also be usefull by the militaries waging this conflict as it will help gauge the risk of
 civilian casualties when plaing an attack.